In [2]:
!wget https://github.com/SirYoja/TaskData/raw/refs/heads/main/KaiNet/best_pictures.tar.gz

--2025-04-30 19:05:25--  https://github.com/SirYoja/TaskData/raw/refs/heads/main/KaiNet/best_pictures.tar.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SirYoja/TaskData/refs/heads/main/KaiNet/best_pictures.tar.gz [following]
--2025-04-30 19:05:25--  https://raw.githubusercontent.com/SirYoja/TaskData/refs/heads/main/KaiNet/best_pictures.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21160383 (20M) [application/octet-stream]
Saving to: ‘best_pictures.tar.gz’

best_pictures.tar.g 100%[===================>]  20.18M  --.-KB/s    in 0.06s   

2025-04-30 19:05:26 (334 MB/s) - ‘best_picture

In [3]:
!tar -xzf best_pictures.tar.gz

tar: Ignoring unknown extended header keyword 'SCHILY.fflags'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.FinderInfo'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'


In [4]:
import torch
from torch import nn
import numpy as np
from matplotlib.image import imread
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold

In [8]:
class DatasetKaiNet(Dataset):
  def __init__(self):
    self.inpictures=[]
    self.outpictures=[]

  def load_pictures(self, number):
    bpath='./'+str(number)+'.png'
    self.inpictures.append(torch.tensor(imread(bpath), dtype=torch.float32).reshape(1,64,64)*255)
    apath='./'+str(number)+'.txt'
    self.outpictures.append(torch.tensor(np.loadtxt(apath), dtype=torch.float32).reshape(1,64,64))

  def __len__(self):
    return len(self.inpictures)

  def __getitem__(self, idx):
    return self.inpictures[idx], self.outpictures[idx]



full_dataset=DatasetKaiNet()
for number in range(1,1001):
  full_dataset.load_pictures(number)



In [9]:
print(imread('1.png')*255)
print(np.loadtxt('1.txt'))

[[174. 160. 172. ... 182. 223. 244.]
 [213. 226. 184. ... 223. 217. 154.]
 [163. 202. 194. ... 165. 127. 135.]
 ...
 [137. 131. 122. ...  99. 100. 114.]
 [136. 129. 118. ...  85.  89. 112.]
 [132. 125. 117. ...  73.  91. 109.]]
[[ 324.81640625  364.234375    353.43359375 ...  326.73828125
   229.59765625   33.4296875 ]
 [ 339.8203125   287.3828125   229.90234375 ...  166.51953125
    54.49609375 -134.234375  ]
 [ 289.40234375  188.03515625  113.96484375 ...   55.84765625
   -46.609375   -207.77734375]
 ...
 [ 227.94921875  131.46875      66.82421875 ...   20.4140625
   -33.46484375 -130.69140625]
 [ 158.03125      54.953125     -8.49609375 ...  -30.85546875
   -69.5234375  -135.56640625]
 [  23.84765625  -83.91796875 -141.98828125 ... -111.99609375
  -124.3828125  -131.65234375]]


In [16]:
train_data_loader=DataLoader(full_dataset, batch_size=1, shuffle=True)

weight1=torch.tensor([[-1.0,-0.5,0.0],[-0.5,0.5,0.5],[0.0,0.5,1.0]])
weight2=torch.tensor([[0.0625,0.0625,0.0625],[0.0625,0.0625,0.0625],[0.0625,0.0625,0.0625]])


weight1=weight1.unsqueeze(0).unsqueeze(0)
weight2=weight2.unsqueeze(0).unsqueeze(0)

In [17]:
class trymodel1(nn.Module):
  def __init__(self):
      super().__init__()
      self.conv1=nn.Conv2d(1,1,3, padding=1, bias=False)
      self.conv2=nn.Conv2d(1,1,3, padding=1, bias=False)
      self.conv3=nn.Conv2d(1,1,3, padding=1, bias=False)

      self.conv3.weight=torch.nn.Parameter(weight1)
      self.conv1.weight=torch.nn.Parameter(weight2)
      self.conv3.weight.requires_grad=False
      self.conv1.weight.requires_grad=False



  def forward(self, x):
      x=self.conv1(x)
      x=self.conv2(x)
      x=self.conv3(x)
      return x

model=trymodel1()
device='cpu'


In [20]:
def train(model, loss_f, opt, train_data_loader, num_epoch):
  for epoch in range(num_epoch):
    train_history=[]
    test_history=[]

    for image, label in tqdm(train_data_loader):
      temp=[]
      temp_test=[]

      opt.zero_grad()
      label_predicted=model(image.to(device))
      loss=loss_f(label_predicted, label.to(device))
      loss.backward()
      opt.step()
      temp.append(loss.item())

    print(np.mean(temp))
    print(epoch+1)
    print('\n')
  return model

loss_f=nn.MSELoss()
opt=torch.optim.Adam(model.parameters(),lr=0.1)
num_epoch=100
model=train(model.to(device), loss_f, opt, train_data_loader,num_epoch)

  0%|          | 0/1000 [00:00<?, ?it/s]

0.0007991400198079646
1




  0%|          | 0/1000 [00:00<?, ?it/s]

0.00021163534256629646
2




  0%|          | 0/1000 [00:00<?, ?it/s]

...

6.844612130407768e-07
98




  0%|          | 0/1000 [00:00<?, ?it/s]

1.327855443378212e-07
99




  0%|          | 0/1000 [00:00<?, ?it/s]

6.506743431091309
100




In [22]:
model.parameters
model.conv2.weight

Parameter containing:
tensor([[[[0.1245, 0.2516, 0.1236],
          [0.2540, 0.5030, 0.2510],
          [0.1239, 0.2505, 0.1186]]]], requires_grad=True)